# Raspberry Pi Camera Capture of Rubik's Cube State

First we import the libraries we need and initialize a camera 'object.'

In [ ]:
import os
from picamera import PiCamera
from picamera.color import Color
from time import sleep

camera = PiCamera()

In [ ]:
# import a bunch of stuff that we'll use to manipulate our images...
import pandas as pd
from skimage.io import imread
from skimage import filters
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
from skimage.measure import block_reduce
import numpy as np

from sklearn.cluster import KMeans

from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
camera.start_preview()
sleep(3)
camera.stop_preview()

*TADA* ... wait, nothing happened.

In [ ]:
camera.hflip = True

In [ ]:
camera.vflip = True

In [ ]:
camera.brightness = 50 # the default is 50, but you can set it to whatever.

How about some text on the image.

In [ ]:
camera.annotate_foreground = Color(1.0,1.0,0.5)

In [ ]:
camera.annotate_text = ""
camera.annotate_text_size = 36

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeU.jpg')
camera.stop_preview()

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeR.jpg')
camera.stop_preview()

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeF.jpg')
camera.stop_preview()

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeD.jpg')
camera.stop_preview()

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeL.jpg')
camera.stop_preview()

In [ ]:
camera.start_preview()
sleep(2)
camera.capture('./img/cubeB.jpg')
camera.stop_preview()

# Once images are captured, let's try to get a 'color' from each square

In [ ]:
face_order = 'URFDLB'

In [ ]:
face_images = {}
face_images_out = []
squares = {}
masks = {}  # for QA

In [ ]:
for face in face_order:
    # read and cache images in dict
    face_images[face] = (imread('./img/cube%s.jpg' % face))

    img = face_images[face]
    mask = np.empty(img.shape[:2], dtype=np.bool)
    mask[::]=False
    row_coords = [(25, 30), (50, 55), (75, 80)]
    col_coords = [(38, 43), (65, 70), (95, 100)]
    squares[face] = np.zeros((3, 3, 3))
    row = 0
    # extract average RGB values from approximate square centers
    for i in row_coords:
        col = 0
        for j in col_coords:
            mask[i[0]:i[1], j[0]:j[1]] = True
            squares[face][row, col] = (np.average(img[i[0]:i[1], j[0]:j[1], 0]),
                                 np.average(img[i[0]:i[1], j[0]:j[1], 1]),
                                 np.average(img[i[0]:i[1], j[0]:j[1], 2]))
            col+=1
        row+=1
    # to show last image alignment
    masks[face] = mark_boundaries(img, mask)

In [ ]:
fig_mask = plt.figure()
ax = fig_mask.add_subplot(1,1,1)
ax.imshow(masks['D'])
plt.axis("off")

In [ ]:
# this basically puts all the average color values for each square into a big table, so we can do some math
square_list = []
for face in face_order:
    for x in range(3):
        for y in range(3):
            r, g, b = squares[face][x,y]
            square_list.append([face, x, y, r, g, b])

columns = ['face', 'x', 'y', 'r', 'g', 'b']
square_frame = pd.DataFrame(square_list, columns=columns)

In [ ]:
square_frame.head(10)

### Our task is to assign each square a 'color.'  To do this, we try to group them (since the computer doesn't really know what 'color' is.

Looking at the colors on a plot, say red vs blue, we get this:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(square_frame.r, square_frame.b, c='gray', marker='o')

ax.set_xlabel('red')
ax.set_ylabel('blue')

Now, how about red vs green vs blue (a 3d cross plot)?

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(square_frame.r, square_frame.g, square_frame.b, c='gray', marker='o')

ax.set_xlabel('red')
ax.set_ylabel('green')
ax.set_zlabel('blue')

In [ ]:
X = square_frame[['r', 'g', 'b']]

In [ ]:
km = KMeans(n_clusters=6, random_state=123)
km.fit(X)

In [ ]:
km.cluster_centers_

In [ ]:
centers_list = []
center_rgb = square_frame[(square_frame['x']==1) & (square_frame['y']==1)]
print center_rgb

In [ ]:
square_mapper = {}
center_map = km.predict(center_rgb[['r', 'g', 'b']])
for i in range(6):
    square_mapper[center_map[i]] = face_order[i]

In [ ]:
center_map

In [ ]:
square_mapper

In [ ]:
square_predict = km.predict(square_frame[['r', 'g', 'b']])

In [ ]:
squares = []
for square in square_predict:
    squares.append(square_mapper[square])

In [ ]:
square_frame['predict'] = squares

In [ ]:
square_color_map = {'U': 'black', 'R': 'red', 'F': 'green', 'D': 'yellow', 'L': 'orange', 'B': 'blue'}

In [ ]:
square_colors = [square_color_map[sq] for sq in squares]

In [ ]:
square_frame['colors'] = square_colors

In [ ]:
square_frame[:15]

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(square_frame.r, square_frame.g, square_frame.b, c=square_frame.colors, marker='o')

ax.set_xlabel('red')
ax.set_ylabel('green')
ax.set_zlabel('blue')